In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Input
import matplotlib.pyplot as plt
from autils import *
%matplotlib inline

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [ ]:
# load dataset
X, y = load_data()

In [ ]:
print("The first element of X is: ", X[0])

In [ ]:
print("The first element of y is: ", y[0, 0])
print("The last element of y is: ", y[-1, 0])

In [ ]:
print("The shape of X is: " + str(X.shape))
print("The shape of y is: " + str(y.shape))

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
# You do not need to modify anything in this cell

m, n = X.shape

fig, axes = plt.subplots(8, 8, figsize=(8, 8))
fig.tight_layout(pad=0.1)

for i, ax in enumerate(axes.flat):
    # Select random indices
    random_index = np.random.randint(m)

    # Select rows corresponding to the random indices and
    # reshape the image
    X_random_reshaped = X[random_index].reshape((20, 20)).T

    # Display the image
    ax.imshow(X_random_reshaped, cmap="gray")

    # Display the label above the image
    ax.set_title(y[random_index, 0])
    ax.set_axis_off()

In [ ]:
model = Sequential(
    [
        Input(shape=(400,)),  # specify input size
        Dense(25, activation="sigmoid", name="layer1"),
        Dense(15, activation="sigmoid", name="layer2"),
        Dense(1, activation="sigmoid", name="layer3"),
    ],
    name="my_model",
)

In [ ]:
model.summary()

In [ ]:
L1_num_params = 400 * 25 + 25  # W1 parameters  + b1 parameters
L2_num_params = 25 * 15 + 15  # W2 parameters  + b2 parameters
L3_num_params = 15 * 1 + 1  # W3 parameters  + b3 parameters
print(
    "L1 params = ",
    L1_num_params,
    ", L2 params = ",
    L2_num_params,
    ",  L3 params = ",
    L3_num_params,
)

In [ ]:
print(model.layers)
[layer1, layer2, layer3] = model.layers

In [ ]:
#### Examine Weights shapes
W1, b1 = layer1.get_weights()
W2, b2 = layer2.get_weights()
W3, b3 = layer3.get_weights()
print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")
print(f"W3 shape = {W3.shape}, b3 shape = {b3.shape}")

In [ ]:
print(W3, b3)
print(model.layers[2].weights)

In [ ]:
model.compile(
    loss=keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(0.001)
)
model.fit(X, y, epochs=20)

In [ ]:
prediction = model.predict(X[0].reshape(1, 400), verbose=0)  # a zero
print(f" predicting a zero: {prediction}")
prediction = model.predict(X[500].reshape(1, 400), verbose=0)  # a one
print(f" predicting a one:  {prediction}")

In [ ]:
prediction = (
    (model.predict(X[0].reshape(1, 400), verbose=0) >= 0.5).astype(int).squeeze()
)  # a zero
print(f" predicting a zero: {prediction}")
prediction = (
    (model.predict(X[500].reshape(1, 400), verbose=0) >= 0.5).astype(int).squeeze()
)  # a one
print(f" predicting a one:  {prediction}")

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

m, n = X.shape

fig, axes = plt.subplots(8, 8, figsize=(8, 8))
fig.tight_layout(pad=0.1, rect=[0, 0.03, 1, 0.92])  # [left, bottom, right, top]

for i, ax in enumerate(axes.flat):
    idx = np.random.randint(m)
    X_idx_reshaped = X[idx].reshape((20, 20)).T
    ax.imshow(X_idx_reshaped, cmap="gray")
    prediction = (
        (model.predict(X[idx].reshape(1, -1), verbose=0) >= 0.5).astype(int).squeeze()
    )
    ax.set_title(f"{y[idx, 0]},{prediction}")
    ax.set_axis_off()
fig.suptitle("Label, Prediction", fontsize=16)
plt.show()

In [ ]:
# UNQ_C2
# GRADED FUNCTION: my_dense


def my_dense(a_in, W, b, g):
    """
    Computes dense layer
    Args:
      a_in (ndarray (n, )) : Data, 1 example
      W    (ndarray (n,j)) : Weight matrix, n features per unit, j units
      b    (ndarray (j, )) : bias vector, j units
      g    activation function (e.g. sigmoid, relu..)
    Returns
      a_out (ndarray (j,))  : j units
    """
    units = W.shape[1]
    a_out = np.zeros(units)
    ### START CODE HERE ###
    for j in range(units):
        w = W[:, j]
        z = np.dot(a_in, w) + b[j]
        a_out[j] = g(z)
    ### END CODE HERE ###
    return a_out

In [ ]:
x_tst = 0.1 * np.arange(1, 3, 1).reshape(
    2,
)  # (1 examples, 2 features)
W_tst = 0.1 * np.arange(1, 7, 1).reshape(2, 3)  # (2 input features, 3 output features)
b_tst = 0.1 * np.arange(1, 4, 1).reshape(
    3,
)  # (3 features)
A_tst = my_dense(x_tst, W_tst, b_tst, sigmoid)
print(A_tst)

In [ ]:
test_c2(my_dense)

In [ ]:
def my_sequential(x, W1, b1, W2, b2, W3, b3):
    a1 = my_dense(x, W1, b1, sigmoid)
    a2 = my_dense(a1, W2, b2, sigmoid)
    a3 = my_dense(a2, W3, b3, sigmoid)
    return a3

In [ ]:
W1_tmp, b1_tmp = layer1.get_weights()
W2_tmp, b2_tmp = layer2.get_weights()
W3_tmp, b3_tmp = layer3.get_weights()

In [ ]:
prediction = (
    (my_sequential(X[0], W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp) >= 0.5)
    .astype(int)
    .squeeze()
)
print(f"prediction = {prediction}, label = {y[0, 0]}")
prediction = (
    (my_sequential(X[500], W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp) >= 0.5)
    .astype(int)
    .squeeze()
)
print(f"prediction = {prediction}, label = {y[500, 0]}")

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

m, n = X.shape

fig, axes = plt.subplots(8, 8, figsize=(8, 8))
fig.tight_layout(pad=0.1, rect=[0, 0.03, 1, 0.92])  # [left, bottom, right, top]
for i, ax in enumerate(axes.flat):
    idx = np.random.randint(m)
    X_idx_reshaped = X[idx].reshape((20, 20)).T
    ax.imshow(X_idx_reshaped, cmap="gray")
    my_prediction = (
        (my_sequential(X[idx], W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp) >= 0.5)
        .astype(int)
        .squeeze()
    )
    tf_prediction = (
        (model.predict(X[idx].reshape(1, -1), verbose=0) >= 0.5).astype(int).squeeze()
    )
    ax.set_title(f"{y[idx, 0]},{tf_prediction},{my_prediction}")
    ax.set_axis_off()
fig.suptitle("Label, Tf Prediction, Numpy Prediction", fontsize=16)
plt.show()

In [ ]:
x = X[0].reshape(-1, 1)  # column vector (400,1)
z1 = np.matmul(x.T, W1) + b1  # (1,400)(400,25) = (1,25)
a1 = sigmoid(z1)
print(a1.shape)

In [ ]:
# UNQ_C3
# UNGRADED FUNCTION: my_dense_v


def my_dense_v(A_in, W, b, g):
    """
    Computes dense layer
    Args:
      A_in (ndarray (m,n)) : Data, m examples, n features each
      W    (ndarray (n,j)) : Weight matrix, n features per unit, j units
      b    (ndarray (1,j)) : bias vector, j units
      g    activation function (e.g. sigmoid, relu..)
    Returns
      A_out (tf.Tensor or ndarray (m,j)) : m examples, j units
    """
    ### START CODE HERE ###
    z = np.matmul(A_in, W) + b
    A_out = g(z)

    ### END CODE HERE ###
    return A_out

In [ ]:
X_tst = 0.1 * np.arange(1, 9, 1).reshape(4, 2)  # (4 examples, 2 features)
W_tst = 0.1 * np.arange(1, 7, 1).reshape(2, 3)  # (2 input features, 3 output features)
b_tst = 0.1 * np.arange(1, 4, 1).reshape(1, 3)  # (1,3 features)
A_tst = my_dense_v(X_tst, W_tst, b_tst, sigmoid)
print(A_tst)

In [ ]:
test_c3(my_dense_v)

In [ ]:
def my_sequential_v(X, W1, b1, W2, b2, W3, b3):
    A1 = my_dense_v(X, W1, b1, sigmoid)
    A2 = my_dense_v(A1, W2, b2, sigmoid)
    A3 = my_dense_v(A2, W3, b3, sigmoid)
    return A3

In [ ]:
W1_tmp, b1_tmp = layer1.get_weights()
W2_tmp, b2_tmp = layer2.get_weights()
W3_tmp, b3_tmp = layer3.get_weights()

In [ ]:
Prediction = my_sequential_v(X, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp)
Prediction.shape

In [ ]:
Yhat = (Prediction >= 0.5).astype(int)
print("predict a zero: ", Yhat[0], "predict a one: ", Yhat[500])

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

m, n = X.shape

fig, axes = plt.subplots(8, 8, figsize=(8, 8))
fig.tight_layout(pad=0.1, rect=[0, 0.03, 1, 0.92])  # [left, bottom, right, top]
for i, ax in enumerate(axes.flat):
    idx = np.random.randint(m)
    X_idx_reshaped = X[idx].reshape((20, 20)).T
    ax.imshow(X_idx_reshaped, cmap="gray")
    ax.set_title(f"{y[idx, 0]},{Yhat[idx, 0]}")
    ax.set_axis_off()
fig.suptitle("Label, Yhat", fontsize=16)
plt.show()

In [ ]:
fig = plt.figure(figsize=(1, 1))
errors = np.where(y != Yhat)
idx = errors[0][0]
X_idx_reshaped = X[idx].reshape((20, 20)).T
plt.imshow(X_idx_reshaped, cmap="gray")
plt.title(f"{y[idx,0]}, {Yhat[idx, 0]}")
plt.axis("off")
plt.show()

In [ ]:
a = np.array([1, 2, 3]).reshape(-1, 1)  # (3,1)
b = 5
print(f"(a + b).shape: {(a + b).shape}, \na + b = \n{a + b}")

In [ ]:
a = np.array([1, 2, 3]).reshape(-1, 1)  # (3,1)
b = 5
print(f"(a * b).shape: {(a * b).shape}, \na * b = \n{a * b}")

In [ ]:
a = np.array([1, 2, 3, 4]).reshape(-1, 1)
b = np.array([1, 2, 3]).reshape(1, -1)
print(a)
print(b)
print(f"(a + b).shape: {(a + b).shape}, \na + b = \n{a + b}")